In [ ]:
pip install flask_mysqldb

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# PREMIER SITE ESSAI

from flask import Flask, render_template, Markup, request, url_for, redirect
#from flask_mail import Mail, Message
from itsdangerous import URLSafeTimedSerializer, SignatureExpired
import cx_Oracle
import datetime
from sqlalchemy import create_engine
from gevent.pywsgi import WSGIServer
from flask_mysqldb import MySQL
import random
from flask import request
from datetime import datetime
import pandas as pd
import numpy as np



app=Flask(__name__)

app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''
app.config['MYSQL_DB'] = 'abdia_bon'

mysql=MySQL(app)

@app.route('/')#index
def index():
    return render_template("pages/index.html")


@app.route('/vote')#index
def vote():
    cur = mysql.connection.cursor()
    strSQL = """ select c.id_categorie, i.id_influenceur
                from categorie c, influenceur i, participer p
                where i.id_categorie=c.id_categorie and p.id_influenceur=i.id_influenceur and p.id_event=1 
                ORDER by c.id_categorie;"""
    
    cur.execute(strSQL)
    mysql.connection.commit()
    fetchdata = cur.fetchall()
    cur.close()

    dico_cat_inf={}
    for tuple in fetchdata:
        if tuple[0] not in list(dico_cat_inf.keys()):
            dico_cat_inf[tuple[0]]=[tuple[1]]
        else:
            dico_cat_inf[tuple[0]].append(tuple[1])
      
    return render_template("pages/vote.html", dico=dico_cat_inf)


@app.route('/inscrit/', methods = ['GET', 'POST'])
def inscrit():
    pseudo = request.form['pseudo']
    adresse_ip = request.remote_addr
    age=request.form['age']
    genre=request.form['genre']
    pays=request.form['pays']
    id_influenceur_voté=int(request.form['vote'])
    categorie_influenceur=int(request.form['categ'])
    date=str(datetime.today().strftime('%Y-%m-%d'))
    print(pays)
    cur = mysql.connection.cursor()
    
    try:
        strSQL = " SELECT `procédure_vote`('"+pseudo+"', '"+adresse_ip+"', '"+age+"', '"+pays+"', '"+genre+"', '1',  '"+str(id_influenceur_voté)+"', '"+date+"') AS `procédure_vote`;"
        print(strSQL)
        cur.execute(strSQL)
        mysql.connection.commit()
        fetchdata = cur.fetchall()
        cur.close()
        retour=fetchdata[0][0]
        print(retour)
    finally:
        cur.close()
        
    return redirect("/visualisation")


@app.route('/visualisation')
def mapbox():
    pays_coor = pd.read_csv('fichiers/pays_avec_coordonnées.csv')
    pays_coor=pays_coor.set_index('NOM')
    
    cur = mysql.connection.cursor()

    select_vote = ("SELECT utilisateur.pays, utilisateur.age, utilisateur.genre FROM voter, utilisateur WHERE voter.pseudo=utilisateur.pseudo and voter.id_event=1 limit 100")
    cur.execute(select_vote)

    result = cur.fetchall()
    print(result[:10])
    cur.close()
    votes = pd.DataFrame(result, columns=['pays', 'age', 'genre'])
    votes_final=votes
    print('laaaaaaa')
    coord=[]
    for i in range (votes_final.shape[0]):
        pays=votes_final.loc[i, 'pays']
        coord.append(list(pays_coor.loc[pays, ['longitude', 'latitude']]))
    
    coord1 = pd.DataFrame(coord, columns=['longitude', 'latitude'])
    mapp = pd.concat([votes_final, coord1], axis = 1)
    
    cols = ['age']
    df=mapp
    properties=cols
    lat='latitude'
    lon='longitude'
    geojson = {'type':'FeatureCollection', 'features':[]}
    i=0
    for _, row in df.iterrows():
        print(i)
        i+=1
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
       
    return render_template('pages/mapboxx.html')


if __name__=='__main__':
    app.run( port=3309, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3309/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Mar/2021 23:50:43] "GET /vote HTTP/1.1" 200 -


Bahreïn
 SELECT `procédure_vote`('cheikh', '127.0.0.1', '-18ans', 'Bahreïn', 'Homme', '1',  '69', '2021-03-25') AS `procédure_vote`;


127.0.0.1 - - [25/Mar/2021 23:51:28] "POST /inscrit/ HTTP/1.1" 302 -


0
(('Suriname', '18-30ans', 'Autre'), ('Tchad', '30-40ans', 'Autre'), ('France', '40-50ans', 'Autre'), ('Indonésie', '40-50ans', 'Femme'), ('Somalie', '30-40ans', 'Homme'), ('Andorre', '18-30ans', 'Femme'), ('Serbie', '18-30ans', 'Homme'), ('Nouvelle-Zélande', '30-40ans', 'Femme'), ('Qatar', '18-30ans', 'Homme'), ('Slovaquie', '+50ans', 'Homme'))
laaaaaaa
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


127.0.0.1 - - [25/Mar/2021 23:51:30] "GET /visualisation HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 23:51:30] "GET /fonts/BebasNeue-Regular.ttf HTTP/1.1" 404 -
127.0.0.1 - - [25/Mar/2021 23:53:26] "GET / HTTP/1.1" 200 -
